## Projet YOLO Gestures
BURDAIRON Florian BLUMET Thomas 5A Polytech Lyon (12/2024)
## Description
L'objectif a été de réutiliser le modèle de réseau de neurones YOLO (de type CNN) qui permet de faire de la classification et de la détection d'objets. Le modèle (en version 8) a été importé préentraîner. Dans notre objectif de réaliser de la détection de gestes, notamment ceux du jeu Pierre-Papier-Ciseaux, nous avons fine-tuné le modèle en le réentraînant (avec différentes valeurs d'epochs notamment, et une valeur de batch fixé à 8).
Pour ce faire, nous avons utilisé le site Roboflow qui permet d'importer des datasets d'images déjà détourées et annotées. Dans notre cas, nous avons importé un dataset existant d'images associé au jeu (cf https://universe.roboflow.com/roboflow-58fyf/rock-paper-scissors-sxsw/dataset/11). 

## Aperçu visuel

### Vidéo de démonstration

[![vidéo démo](https://img.youtube.com/vi/ReloVy038hk/0.jpg)](https://www.youtube.com/embed/ReloVy038hk?si=sfJW1PBMoYLW4kXn)

### Exemple classification

| Gesture                       | Image                                                                         |
|-------------------------------|-------------------------------------------------------------------------------|
| Paper                         | <img src="img/paper_detection.png" alt="paper_detection" width="500px">       |
| Rock                          | <img src="img/rock_detection.png" alt="rock_detection" width="500px">         |
| Scissors                      | <img src="img/scissors_detection.png" alt="scissors_detection" width="500px"> |
| 2 detections at the same time | <img src="img/round.png" alt="round" width="500px">                           |

## Lancement du projet

> Remarque : \
> Le fichier [`YOLO_webcam.py`](YOLO_webcam.py) contient l'entièreté du code du projet permettant une exécution simplifiée.

### Besoin du projet

Pour ce projet, nous avons utilisé plusieurs librairies python :
- Ultralytics : pour l'utilisation de YOLO
- Roboflow : pour l'importation du jeu de données
- Torch : pour l'entrainement du modèle (avec CUDA)
- OpenCV : pour l'utilisation de la webcam
- YAML : pour la lecture du fichier de description du jeu de données

> Remarque : \
> Pour réaliser l'entraînement, il faut importer la librairie Torch CUDA et posséder une carte graphique Nvidia. \
> L'utilisation des modèles que nous avons déjà entrainé (`model/`) ne nécessite pas l'utilisation de CUDA.


In [7]:
# Import libraries 
from ultralytics import YOLO, settings
import cv2
import math 
import torch
import os
from roboflow import Roboflow
import yaml

# Select the device to be used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Select the directory where the datasets are stored
settings.update(datasets_dir='.')


### Importation du jeu de données et entrainement du modèle

Voici une description des fonctions que nous utilisons dans le projet :
- `train_and_save_model` : Permet d'entrainer le modèle sur un jeu de données spécifique (`model_name`) et de sauvegarder les points du réseau. Il est possible de choisir des paramètre pour l'entrainement (`nb_epochs` et `batch_size`).
- `get_class_names` : Permet de récupérer la liste des éléments classifiables contenue dans le fichier de description du jeu de données `data.yaml`.
- `load_model_from_roboflow` : Permet de télécharger le jeu de données directement depuis Roboflow s'il n'est pas déjà dans `data/` puis d'entrainer le modèle si celui-ci n'a as déjà été entrainer dessus.
- `load_model` : Permet de récuperer le modèle correspondant a un jeu de données déjà présent dans `data/` et de l'entrainer si il ne l'a pas déjà été. (Non utilisé)

In [2]:
def train_and_save_model(model_name,nb_epochs=10,batch_size=8) -> YOLO:
    """
    Train and save model

    Model will be saved in `model/` directory
    """
    # Initialize model with pre-trained weights (YOLO v8)
    model = YOLO("yolo-Weights/yolov8n.pt")

    # Train model on custom dataset
    model.train(data="data/" + model_name + "/data.yaml", epochs=nb_epochs, batch=batch_size, device=device)

    # Save model in `model/` directory
    model.save("model/" + model_name + "_e" + str(nb_epochs) + "_b" + str(batch_size) + ".pt")

    return model

def get_class_names(yaml_path):
    """Get class names from yaml file"""
    with open(yaml_path, 'r') as file:
        data = yaml.safe_load(file)
    return data['names']


def load_model_from_roboflow(workspace, project_name, version_number, model_name,nb_epochs,batch_size) -> tuple[YOLO, any]:
    """
    Load model from Roboflow

    If the dataset is not in `data/` directory, then it will be downloaded

    If the model is not in `model/` directory, then it will be trained and its weights will be saved
    """
    # Download dataset if not exists
    if not os.path.exists("data/" + model_name):
        rf = Roboflow(api_key="8DrZ8Cjqqu2mLaJM9iPH")
        project = rf.workspace(workspace).project(project_name)
        version = project.version(version_number)
        dataset = version.download("yolov8", "data/" + model_name)
    
    # Get class names from yaml file
    classNames = get_class_names("data/" + model_name + "/data.yaml")

    # Load model if exists
    if os.path.exists("model/" + model_name + "_e" + str(nb_epochs) + "_b" + str(batch_size) + ".pt"):
        return YOLO("model/" + model_name + "_e" + str(nb_epochs) + "_b" + str(batch_size) + ".pt"), classNames
    
    # If model does not exist, then train and save it
    return train_and_save_model(model_name,nb_epochs,batch_size), classNames

def load_model(model_name,nb_epochs,batch_size) -> tuple[YOLO, any]:
    """
    Load model
    
    The dataset should be in `data/` directory

    If the model is not in `model/` directory, then it will be trained and its weights will be saved
    """
    # Get class names from yaml file
    classNames = get_class_names("data/" + model_name + "/data.yaml")

    # Load model if exists
    if os.path.exists("model/" + model_name + "_e" + str(nb_epochs) + "_b" + str(batch_size) + ".pt"):
        model = YOLO("model/" + model_name + "_e" + str(nb_epochs) + "_b" + str(batch_size) + ".pt")
    # If model does not exist, then train and save it
    else:
        model = train_and_save_model(model_name,nb_epochs,batch_size)

    return model, classNames

In [ ]:
# For the example, load the dataset from Roboflow and train the model with 5 epochs and batch size of 8
def example() -> tuple[YOLO, any]:
    """
    Get model

    If the dataset is not in `data/` directory, then it will be downloaded

    If the model is not in `model/` directory, then it will be trained and its weights will be saved
    """
    model_name = "rock-paper-scissors"
    print("Loading model " + model_name + "...")

    # Can replace the following line with other parameters to match the trained model from `model/` directory to avoid the trainig process
    return load_model_from_roboflow("roboflow-58fyf", "rock-paper-scissors-sxsw", 11, model_name, 5, 8) # workspace, project_name, version_number, model_name, nb_epochs, batch_size
    # return load_model_from_roboflow("roboflow-58fyf", "rock-paper-scissors-sxsw", 11, model_name, 50, 8) # Best model

if __name__ == "__main__":
    # Clear cuda cache before running the code
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    model, classNames = example()
    print("Model loaded successfully")
    print("Class names: ", classNames)

Loading model rock-paper-scissors...
New https://pypi.org/project/ultralytics/8.3.49 available  Update with 'pip install -U ultralytics'
engine\trainer: task=detect, mode=train, model=yolo-Weights/yolov8n.pt, data=data/rock-paper-scissors/data.yaml, epochs=5, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

train: Scanning D:\PolytechTP5AHandGestureGames\data\rock-paper-scissors\train\labels.cache... 14966 images, 5957 backgrounds, 0 corrupt: 100%|██████████| 14966/14966 [00:00<?, ?it/s]
val: Scanning D:\PolytechTP5AHandGestureGames\data\rock-paper-scissors\valid\labels.cache... 588 images, 247 backgrounds, 0 corrupt: 100%|██████████| 588/588 [00:00<?, ?it/s]


Plotting labels to runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train8
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5       1.2G      1.455       2.74      1.611          8        640: 100%|██████████| 1871/1871 [01:46<00:00, 17.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:01<00:00, 20.28it/s]

                   all        588        406      0.581      0.506      0.539      0.259


...

        5/5      1.24G      1.195      1.145      1.376          7        640: 100%|██████████| 1871/1871 [01:34<00:00, 19.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:01<00:00, 21.12it/s]

                   all        588        406      0.939      0.888      0.943      0.558



5 epochs completed in 0.139 hours.
Optimizer stripped from runs\detect\train8\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train8\weights\best.pt, 6.2MB

Validating runs\detect\train8\weights\best.pt...
Ultralytics 8.3.24  Python-3.12.4 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 37/37 [00:01<00:00, 21.89it/s]


                   all        588        406      0.939      0.888      0.943      0.558
                 Paper        134        141      0.903      0.857      0.929      0.526
                  Rock        125        147       0.94      0.925      0.945      0.544
              Scissors        114        118      0.974      0.881      0.954      0.604
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train8
Model loaded successfully
Class names:  ['Paper', 'Rock', 'Scissors']


### Lancement du jeu

Quand le jeu démarre, une fenêtre apparaît avec la webcam et une interface séparant la fenêtre en deux. Chaque côté correspond à un joueur et affiche son score et le geste qu'il a choisi durant la manche.

Une manche dure 50 frames. Cette durée est représentée par la barre de progression en bas de la fenêtre. À la fin de chaque manche, un vainqueur est désigné de la manière suivante :
- Si un des 2 joueurs n'a fait aucun geste, aucun vainqueur n'est désigné.
- Si les deux joueurs ont fait le même geste, aucun vainqueur n'est désigné.
- Sinon les règles classiques du **Pierre Papier Ciseaux** s'appliquent (Pierre bat Ciseaux, Ciseaux bat Papier et Papier bat Pierre).

Pour faire un geste le joueur doit faire un geste dans sa partie de l'écran. Il a ensuite une marge de manoeuvre de 20 frames pour changer son geste (cela sert à éviter qu'une détection soit réalisé avant que le joueur finisse de faire son geste).

Nous avons choisi de faire des manches assez courtes pour éviter la triche (un joueur attend que l'autre joue pour connaître son geste avant de jouer). Le fait de fixer le choix d'un joueur permet d'éviter qu'il change de geste pendant la manche (de manière volontaire pour tricher ou de manière involontaire dûe à une erreur de classification).

> Pour quitter le jeu, il faut appuyer sur la touche `q`.

In [9]:
# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

# Initialize game variables
score_player1 = 0
score_player2 = 0
sign_player1 = ""
sign_player2 = ""
player1_first_detect = 100
player2_first_detect = 100

frame_count = 0

# Start game loop
while cap.isOpened():
    # Capture a frame
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        continue

    # Mirror the image horizontally
    img = cv2.flip(img, 1)
    
    frame_count += 1

    # Detect objects in the image using YOLO
    results = model(img, stream=True)

    # Build UI with scores and signs
    cv2.rectangle(img, (0, 0), (637, 720), (0, 0, 255), 3)
    cv2.rectangle(img, (642, 0), (1280, 720), (255, 0, 0), 3)
    cv2.putText(img, "Player 1: " + str(score_player1), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img, "Player 2: " + str(score_player2), (652, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img, sign_player1, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
    cv2.putText(img, sign_player2, (652, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Display progress bar for 50 frames to show the time limit for the round
    cv2.rectangle(img, (0, 700), (round(frame_count / 50 * 1280), 720), (0, 255, 0), -1)

    # Loop through the results and draw bounding boxes
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Get bounding box coordinates
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # Get confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # Get class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # Get the mean of x coordinates
            x_mean = (x1 + x2) / 2

            # Check if the object is on the left or right side of the screen
            if x_mean < 640:
                # Player 1
                # Check if the sign is detected for the first time or not
                if sign_player1 == "":
                    player1_first_detect = frame_count
                    sign_player1 = classNames[cls].lower()
                # Check if the sign is detected again within 20 frames
                elif frame_count - player1_first_detect < 20:
                    sign_player1 = classNames[cls].lower()
                else:
                    continue
            else:
                # Player 2
                # Check if the sign is detected for the first time or not
                if sign_player2 == "":
                    player2_first_detect = frame_count
                    sign_player2 = classNames[cls].lower()
                # Check if the sign is detected again within 20 frames
                elif frame_count - player2_first_detect < 20:
                    sign_player2 = classNames[cls].lower()
                else:
                    continue

            # Put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Put object details in cam
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2
            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    # Check if the round is over
    if(frame_count >= 50):
        # Check who wins the round and update the scores
        if sign_player1 == "rock" and sign_player2 == "scissors":
            score_player1 += 1
        elif sign_player1 == "scissors" and sign_player2 == "rock":
            score_player2 += 1
        elif sign_player1 == "scissors" and sign_player2 == "paper":
            score_player1 += 1
        elif sign_player1 == "paper" and sign_player2 == "scissors":
            score_player2 += 1
        elif sign_player1 == "rock" and sign_player2 == "paper":
            score_player2 += 1
        elif sign_player1 == "paper" and sign_player2 == "rock":
            score_player1 += 1

        # Reset the signs and frame count for the next round
        sign_player1 = ""
        sign_player2 = ""
        player1_first_detect = 100
        player2_first_detect = 100
        frame_count = 0

    # Display the image
    cv2.imshow('Rock Paper Scissors Game', img)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 33.6ms
Speed: 1.5ms preprocess, 33.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 4.7ms
Speed: 1.0ms preprocess, 4.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.2ms
Speed: 0.7ms preprocess, 5.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 4.4ms
Speed: 0.5ms preprocess, 4.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 3.6ms
Speed: 1.5ms preprocess, 3.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 4.2ms
Speed: 0.0ms preprocess, 4.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.5ms
Speed: 0.0ms preprocess, 5.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.5m